In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs

import requests
import json
from random import randint
import os
import time
import json
import pickle

In [6]:
detalles

{110: {'detalles': {'id': 110,
   'title': 'Chuuka Ichiban!',
   'main_picture': {'medium': 'https://cdn.myanimelist.net/images/anime/6/75283.jpg',
    'large': 'https://cdn.myanimelist.net/images/anime/6/75283l.jpg'},
   'alternative_titles': {'synonyms': ['Cooking Master Boy'],
    'en': '',
    'ja': '中華一番'}}},
 128: {'detalles': {'id': 128,
   'title': 'Gate Keepers 21',
   'main_picture': {'medium': 'https://cdn.myanimelist.net/images/anime/6/75569.jpg',
    'large': 'https://cdn.myanimelist.net/images/anime/6/75569l.jpg'},
   'alternative_titles': {'synonyms': ['Gatekeepers 21'],
    'en': '',
    'ja': 'ゲートキーパーズ21'}}},
 134: {'detalles': {'id': 134,
   'title': 'Gunslinger Girl',
   'main_picture': {'medium': 'https://cdn.myanimelist.net/images/anime/5/3088.jpg',
    'large': 'https://cdn.myanimelist.net/images/anime/5/3088l.jpg'},
   'alternative_titles': {'synonyms': ['Gunslinger Girls'],
    'en': 'Gunslinger Girl',
    'ja': 'ガンスリンガー・ガール'}}},
 159: {'detalles': {'id': 159,
 